<a href="https://colab.research.google.com/github/GenaroHacker/creating_chord_collection/blob/main/reading_the_chords.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:


%%capture
#@title Set Up
!git clone https://github.com/GenaroHacker/write_imports.git
from write_imports.write_imports import write_imports

!git clone https://github.com/GenaroHacker/creating_chord_collection.git


from creating_chord_collection.collection.resources.scales import scales
from creating_chord_collection.collection.resources.transposable_figures import transposable_figures





In [36]:

# Delete this line to see the new imports
%%capture



#@title Import Statements
#Modules: ['creating_chord_collection', 'write_imports']




from creating_chord_collection.builder.director import Director

from creating_chord_collection.builder.builders import AbstractBuilder
from creating_chord_collection.builder.builders import ShortBuilder
from creating_chord_collection.builder.builders import LongBuilder

from creating_chord_collection.collection.chord import GuitarChord

from creating_chord_collection.collection.collection import ChordCollection



from write_imports.write_imports import write_imports

from write_imports.LearningPython.BasicExamples.intro_functions import print_hello_world





print(write_imports([]))


In [37]:
# @title txt to dict
def parse_chords(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    chords = {}
    current_chord = None

    for line in lines:
        line = line.strip()
        if not line:
            continue

        # Check if the line is a chord name (no spaces) or a chord position (contains spaces)
        if ' ' in line:
            if current_chord:
                chords[current_chord].append(line)
        else:
            current_chord = line
            chords[current_chord] = []

    return chords

# Example usage
file_path = '/content/creating_chord_collection/chords.txt'  # Replace with your file path
chord_dict = parse_chords(file_path)
print(chord_dict['A'][0:2])
print(chord_dict['Gbmmaj11/A'][0:2])


['x 0 2 2 2 0', '5 7 7 6 5 5']
['x 0 3 2 0 2', '5 4 4 4 6 4']


In [38]:
# @title Split by / character
def process_keys(my_dict):
    absolute_keys_dict = {}
    left_relative_keys_dict = {}
    right_relative_keys_dict = {}

    for key, value in my_dict.items():
        if '/' in key:
            parts = key.split('/')
            if len(parts) != 2:
                raise ValueError("Key contains more than one '/' character: " + key)
            left_relative_keys_dict[parts[0]] = value
            right_relative_keys_dict[parts[1]] = value
        else:
            absolute_keys_dict[key] = value

    return absolute_keys_dict, left_relative_keys_dict, right_relative_keys_dict

# Example usage
my_dict = chord_dict

try:
    abs_keys_dict, left_rels_dict, right_rels_dict = process_keys(my_dict)
    #print("Absolute Keys Dict:", abs_keys_dict)
    #print("Left Parts of Relative Keys Dict:", left_rels_dict)
    #print("Right Parts of Relative Keys Dict:", right_rels_dict)
except ValueError as e:
    print("Error:", e)


In [39]:
# @title Merge into one dict
def merge_dicts(dict_list):
    merged_dict = {}
    for d in dict_list:
        for key, value in d.items():
            if key not in merged_dict:
                merged_dict[key] = set()
            merged_dict[key].update(value)

    # Convert sets back to lists
    for key in merged_dict:
        merged_dict[key] = list(merged_dict[key])

    return merged_dict

# Example usage
dict_list = [abs_keys_dict, left_rels_dict, right_rels_dict]

merged = merge_dicts(dict_list)
print(merged["G"])


['x 10 9 12 x 10', '3 2 4 1 x 1', 'x 11 12 12 12 x', '4 5 5 7 x 7', 'x 10 12 12 12 10', '3 2 5 x 3 x', 'x 11 9 12 x 10', 'x 11 x 12 12 10', 'x 10 9 7 x 10', '3 5 5 4 3 3', 'x 10 9 x 8 10', '4 x 5 4 3 3', '3 x 3 1 0 2', '4 5 x 4 3 3', '4 2 5 x 3 3', 'x x 6 4 3 3', '4 2 x 4 3 3', '3 2 x 4 3 x', 'x x 5 7 8 7', '15 14 12 12 15 x', '15 14 12 12 12 15', '3 4 4 4 6 4', '3 2 0 0 3 3', '4 5 5 4 x 7', 'x 11 9 x 8 10', 'x x 6 7 8 7', '3 2 0 0 0 3', 'x 10 9 x 12 10', 'x 10 9 7 8 7']


In [40]:
# @title Update dict
def convert_and_invert(input_string):
    elements = input_string.split()
    converted_elements = [None if el == 'x' else int(el) for el in elements]
    inverted_elements = converted_elements[::-1]
    return inverted_elements

def process_dict(input_dict):
    processed_dict = {}
    for key, value in input_dict.items():
        # Apply convert_and_invert to each string in the list
        processed_dict[key] = [convert_and_invert(s) for s in value]

    return processed_dict

# Example usage
my_dict = merged

processed_dict = process_dict(my_dict)
#print(processed_dict)


In [41]:
# @title Remove greater than 12 fret
def remove_sublists_with_large_ints(d):
    for key in d.keys():
        # Remove sublists if any integer inside it is greater than 12
        d[key] = [sublist for sublist in d[key] if all(x is None or (isinstance(x, int) and x <= 12) for x in sublist)]
    return d

# Example usage
data = processed_dict
result = remove_sublists_with_large_ints(data)
print(result)


{'A': [[5, 5, None, None, 4, 5], [None, 5, 2, 2, 4, 5], [12, None, 9, 11, 12, None], [5, 5, 6, 7, 4, 4], [12, 10, None, 11, 12, None], [0, 2, 2, 2, 0, None], [None, 5, 6, None, 4, 5], [5, 2, 2, 2, 4, 4], [5, 5, 2, 2, 4, 4], [None, 5, None, 7, 4, 5], [5, 5, 6, 6, None, None], [None, None, 6, 7, 7, 4], [5, 5, 6, 7, 7, 5], [2, 0, 1, 3, 0, None], [7, 7, 6, 6, 8, 5], [9, 10, 9, 7, None, None], [9, 10, 9, 11, 12, None], [9, 10, 9, 11, 11, None], [5, 2, 2, 2, 4, 5], [12, 10, None, 11, 11, None], [4, 6, 4, 4, 4, 5], [5, 5, 6, None, 7, 4]], 'B': [[2, None, 4, 1, 2, None], [None, 7, 6, 6, 8, 7], [4, 6, 6, 4, 4, 7], [4, 6, 4, 4, 4, 7], [2, None, 4, 1, None, 4], [None, 7, 4, 4, 6, 7], [9, 9, 10, 9, 9, 7], [11, None, 11, 9, 11, None], [2, 4, 4, 4, 2, None], [2, 0, 1, 3, 2, None], [2, 4, None, 4, 2, 4], [7, 4, 4, 4, 6, 7], [7, 9, 10, None, 9, 7], [1, 2, 1, 4, 2, None], [2, 0, None, 1, 2, None], [7, 7, None, None, 6, 7], [7, 7, 4, 4, 6, 4], [9, 6, 6, 6, 9, 7], [7, 7, 8, 6, None, None], [7, 4, 4, 4, 6

In [42]:
# @title split keys

#the function will split the key of the chords dict into a tuple where the first element is the root and the second is the chord type
def split_keys(chords_dict):
    roots = ['Db', 'Eb', 'Gb', 'Ab', 'Bb', 'C#', 'D#', 'F#', 'G#', 'A#', 'C', 'D', 'E', 'F', 'G', 'A', 'B']

    updated_dict = {}

    for key in chords_dict:
        for root in roots:
            if root in key:
                updated_dict[(root, key.replace(root, ''))] = chords_dict[key]
                break
    return updated_dict


splitted = split_keys(result)
print(splitted)

{('A', ''): [[5, 5, None, None, 4, 5], [None, 5, 2, 2, 4, 5], [12, None, 9, 11, 12, None], [5, 5, 6, 7, 4, 4], [12, 10, None, 11, 12, None], [0, 2, 2, 2, 0, None], [None, 5, 6, None, 4, 5], [5, 2, 2, 2, 4, 4], [5, 5, 2, 2, 4, 4], [None, 5, None, 7, 4, 5], [5, 5, 6, 6, None, None], [None, None, 6, 7, 7, 4], [5, 5, 6, 7, 7, 5], [2, 0, 1, 3, 0, None], [7, 7, 6, 6, 8, 5], [9, 10, 9, 7, None, None], [9, 10, 9, 11, 12, None], [9, 10, 9, 11, 11, None], [5, 2, 2, 2, 4, 5], [12, 10, None, 11, 11, None], [4, 6, 4, 4, 4, 5], [5, 5, 6, None, 7, 4]], ('B', ''): [[2, None, 4, 1, 2, None], [None, 7, 6, 6, 8, 7], [4, 6, 6, 4, 4, 7], [4, 6, 4, 4, 4, 7], [2, None, 4, 1, None, 4], [None, 7, 4, 4, 6, 7], [9, 9, 10, 9, 9, 7], [11, None, 11, 9, 11, None], [2, 4, 4, 4, 2, None], [2, 0, 1, 3, 2, None], [2, 4, None, 4, 2, 4], [7, 4, 4, 4, 6, 7], [7, 9, 10, None, 9, 7], [1, 2, 1, 4, 2, None], [2, 0, None, 1, 2, None], [7, 7, None, None, 6, 7], [7, 7, 4, 4, 6, 4], [9, 6, 6, 6, 9, 7], [7, 7, 8, 6, None, None], [7

In [43]:
# @title Def how to transpose a figure
def process_list(input_list):
    # Check if the list contains a 0
    if 0 in input_list:
        # Check if any integer in the list is greater than 4
        if any(isinstance(item, int) and item > 4 for item in input_list):
            raise ValueError("The list contains 0 and an integer greater than 4, which should not happen.")
        else:
            # Return a tuple with 1 and the original list
            return 1, input_list
    else:
        # Filter out None values and find the highest and lowest integers
        filtered_list = [item for item in input_list if isinstance(item, int)]
        if filtered_list:  # Check if filtered_list is not empty
            highest = max(filtered_list)
            lowest = min(filtered_list)
            # Check if the difference is greater than 3
            if highest - lowest > 3:
                raise ValueError("The difference between the highest and lowest integer is greater than 3.")

        # Initialize transpositions
        transpositions = 1
        # Modify the list
        while lowest > 1:
            transpositions += 1
            input_list = [item - 1 if isinstance(item, int) else item for item in input_list]
            filtered_list = [item for item in input_list if isinstance(item, int)]
            if filtered_list:  # Check if filtered_list is not empty
                lowest = min(filtered_list)

        return transpositions, input_list

# Example usage
example_list = [0, 4, 0, 0, 0, None]
process_list(example_list)



(1, [0, 4, 0, 0, 0, None])

In [44]:
# @title transpose all items
def process_my_dict(my_dict):
    output_dict = {}

    for key, value_list in my_dict.items():
        updated_value_list = []

        for sublist in value_list:
            try:
                processed_sublist = process_list(sublist)
                if processed_sublist is not None:
                    updated_value_list.append(processed_sublist)
            except Exception as e:
                # An error occurred in process_list, skipping this sublist
                continue

        output_dict[key] = updated_value_list

    return output_dict


result_dict = process_my_dict(splitted)

In [45]:
# @title create chord list


# Example all_chords dictionary
all_chords = result_dict

guitar_chord_objects = []

for chord_key, variations in all_chords.items():
    root, chord_type = chord_key
    for starting_fret, finger_ascending in variations:
        # Assuming transposable_figures is either not required or can be determined here
        transposable_figures = None
        guitar_chord_objects.append(GuitarChord(root, chord_type, transposable_figures, finger_ascending=finger_ascending, starting_fret=starting_fret))

# Now guitar_chord_objects contains all the GuitarChord instances


In [46]:
print(guitar_chord_objects[0])

('A', '', finger_ascending=[2, 2, None, None, 1, 2], starting_fret=4)


In [47]:
print(transposable_figures)

None


In [48]:
# @title Collection



# Example usage
chord_collection = ChordCollection()
chord_collection.load('/content/creating_chord_collection/collection/resources/chord_collection.db')


chord_collection.chords = guitar_chord_objects
chord = chord_collection.chords[6]
print(chord)
chord.transpose(1)
print(chord)
chord_collection.save('new_chord_collection.db')
print(f"Collection Size: {len(chord_collection.chords)}")
chord_collection.extend_barre_chords()
print(f"Collection Size: {len(chord_collection.chords)}")

whitelist = {
     "root": ["C", "F", "G"],
     "chord_type": ["", "m", "dim"],
     "open": [False],
     "starting_fret": [1, 2, 5, 9],
     "include_string": [1, 2],
     "inversion": [1, 2, 3, None],
     "scale": [("C", scales["ionian"])]
 }

filtered_chords = chord_collection.only(whitelist)
print(f"Remaining after whitelist filter: {len(filtered_chords)}")

blacklist = whitelist
filtered_chords = chord_collection.filter_out(blacklist)
print(f"Remaining after blacklist filter: {len(filtered_chords)}")

tonality = chord_collection.get_tonality("B", scales["aeolian"], amplitude=3, rank=2)
print(f"Chords in tonality: {len(tonality)}")

('A', '', finger_ascending=[None, 2, 3, None, 1, 2], starting_fret=4)


TypeError: argument of type 'NoneType' is not iterable